In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.structured import *
from fastai.column_data import *
from torch.nn import functional as F

In [3]:
print(torch.cuda.is_available())

x = torch.rand(5, 3)
y = torch.rand(5, 3)
x = x.cuda()
y = y.cuda()
x + y

True



 0.6442  0.0847  1.0253
 0.5889  0.8890  0.1898
 0.8223  1.7587  0.7341
 0.6780  0.8726  0.7073
 1.0423  0.7254  1.2464
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
class MixedInputModel(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                 y_range=None, use_bn=False):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)
        self.n_emb, self.n_cont=n_emb, n_cont
        
        szs = [n_emb+n_cont] + szs
        self.lins = nn.ModuleList([
            nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: kaiming_normal(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        kaiming_normal(self.outp.weight.data)

        self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        if self.y_range:
            x = F.log_softmax(x)
            x = x*(self.y_range[1] - self.y_range[0])
            x = x+self.y_range[0]
        return x

In [6]:
class ColumnarDataset(Dataset):
    def __init__(self, cats, conts, y):
        n = len(cats[0]) if cats else len(conts[0])
        self.cats = np.stack(cats, 1).astype(np.int64) if cats else np.zeros((n,1))
        self.conts = np.stack(conts, 1).astype(np.float32) if conts else np.zeros((n,1))
        self.y = np.zeros((n,1)) if y is None else y.values # THIS LINE IS CHANGED FROM y[:, None]
        
    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]

    @classmethod
    def from_data_frames(cls, df_cat, df_cont, y=None):
        cat_cols = [c.values for n,c in df_cat.items()]
        cont_cols = [c.values for n,c in df_cont.items()]
        return cls(cat_cols, cont_cols, y)

    @classmethod
    def from_data_frame(cls, df, cat_flds, y=None):
        return cls.from_data_frames(df[cat_flds], df.drop(cat_flds, axis=1), y)

In [7]:
class ColumnarModelData(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, test_ds=None, shuffle=True):
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, DataLoader(trn_ds, bs, shuffle=shuffle, num_workers=1),
            DataLoader(val_ds, bs*2, shuffle=False, num_workers=1), test_dl)

    @classmethod
    def from_arrays(cls, path, val_idxs, xs, y, bs=64, test_xs=None, shuffle=True):
        ((val_xs, trn_xs), (val_y, trn_y)) = split_by_idx(val_idxs, xs, y)
        test_ds = PassthruDataset(*(test_xs.T), [0] * len(test_xs)) if test_xs is not None else None
        return cls(path, PassthruDataset(*(trn_xs.T), trn_y), PassthruDataset(*(val_xs.T), val_y),
                   bs=bs, shuffle=shuffle, test_ds=test_ds)

    @classmethod
    def from_data_frames(cls, path, trn_df, val_df, trn_y, val_y, cat_flds, bs, test_df=None):
        test_ds = ColumnarDataset.from_data_frame(test_df, cat_flds) if test_df is not None else None
        return cls(path, ColumnarDataset.from_data_frame(trn_df, cat_flds, trn_y),
                    ColumnarDataset.from_data_frame(val_df, cat_flds, val_y), bs, test_ds=test_ds)

    @classmethod
    def from_data_frame(cls, path, val_idxs, df, y, cat_flds, bs, test_df=None):
        ((val_df, trn_df), (val_y, trn_y)) = split_by_idx(val_idxs, df, y)
        return cls.from_data_frames(path, trn_df, val_df, trn_y, val_y, cat_flds, bs, test_df=test_df)

    def get_learner(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                    y_range=None, use_bn=False, **kwargs):
        model = MixedInputModel(emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range, use_bn)
        return StructuredLearner(self, StructuredModel(to_gpu(model)), opt_fn=optim.Adam, **kwargs)

In [8]:
PATH = 'data/biaobin/'
data_txt = f'{PATH}/mmm_p15_m8.txt'

In [9]:
df = pd.read_table(data_txt)
df.shape

(9317, 494)

In [10]:
df.head()

,Lung,Breast,Colon,Central_Nervous_System,Pancreas,Ovary,Prostate,Uterus,Kidney,Head_and_Neck,Stomach,Bladder,Liver,Skin,Thyroid,mut_ABL1,mut_AKT1,mut_AKT2,mut_AKT3,mut_ALK,mut_ALOX12B,mut_AMER1,mut_APC,mut_AR,mut_ARAF,mut_ARID1A,mut_ARID2,mut_ASXL1,mut_ATM,mut_ATR,mut_ATRX,mut_AURKA,mut_AURKB,mut_AXL,mut_BAP1,mut_BARD1,mut_BCL2,mut_BCL6,mut_BCOR,mut_BLM,mut_BRAF,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_BTK,mut_CARD11,mut_CASP8,mut_CBFB,mut_CBL,mut_CCND1,mut_CCND2,mut_CCND3,mut_CCNE1,mut_CD79A,mut_CD79B,mut_CDC73,mut_CDH1,mut_CDK12,mut_CDK4,mut_CDK6,mut_CDK8,mut_CDKN1B,mut_CDKN2A,mut_CDKN2B,mut_CDKN2C,mut_CEBPA,mut_CHEK1,mut_CHEK2,mut_CIC,mut_CREBBP,mut_CRKL,mut_CRLF2,mut_CSF1R,mut_CTCF,mut_CTNNB1,mut_DAXX,mut_DDR2,mut_DIS3,mut_DNMT3A,mut_DOT1L,mut_EGFR,mut_EP300,mut_EPHA3,mut_EPHA5,mut_EPHB1,mut_ERBB2,mut_ERBB3,mut_ERBB4,mut_ERG,mut_ESR1,mut_EZH2,mut_FAM46C,mut_FANCA,mut_FANCC,mut_FBXW7,mut_FGF19,mut_FGF3,mut_FGF4,mut_FGFR1,mut_FGFR2,mut_FGFR3,mut_FGFR4,mut_FIP1L1,mut_FLT1,mut_FLT3,mut_FLT4,mut_FOXL2,mut_GATA1,mut_GATA2,mut_GATA3,mut_GNA11,mut_GNAQ,mut_GNAS,mut_GRIN2A,mut_GSK3B,mut_HGF,mut_HLA_A,mut_HRAS,mut_IDH1,mut_IDH2,mut_IGF1,mut_IGF1R,mut_IGF2,mut_IKBKE,mut_IKZF1,mut_IL7R,mut_INHBA,mut_INSRR,mut_IRF4,mut_IRS2,mut_JAK1,mut_JAK2,mut_JAK3,mut_JUN,mut_KDM5A,mut_KDM5C,mut_KDM6A,mut_KDR,mut_KEAP1,mut_KIT,mut_KMT2A,mut_KMT2D,mut_KRAS,mut_LMO1,mut_MAP2K1,mut_MAP2K2,mut_MAP2K4,mut_MAP3K1,mut_MAP3K13,mut_MCL1,mut_MDM2,mut_MDM4,mut_MED12,mut_MEF2B,mut_MEN1,mut_MET,mut_MITF,mut_MLH1,mut_MPL,mut_MRE11A,mut_MSH2,mut_MSH6,mut_MTOR,mut_MUTYH,mut_MYC,mut_MYCL,mut_MYCN,mut_MYD88,mut_NBN,mut_NCOR1,mut_NF1,mut_NF2,mut_NFE2L2,mut_NFKBIA,mut_NKX2_1,mut_NOTCH1,mut_NOTCH2,mut_NOTCH3,mut_NOTCH4,mut_NPM1,mut_NRAS,mut_NSD1,mut_NTRK1,mut_NTRK2,mut_NTRK3,mut_NUP93,mut_PAK7,mut_PALB2,mut_PARP1,mut_PAX5,mut_PBRM1,mut_PDGFRA,mut_PDGFRB,mut_PDPK1,mut_PIK3C2G,mut_PIK3C3,mut_PIK3CA,mut_PIK3CG,mut_PIK3R1,mut_PIK3R2,mut_PMS2,mut_PNRC1,mut_PPP2R1A,mut_PRDM1,mut_PRKAR1A,mut_PTCH1,mut_PTEN,mut_PTPN11,mut_RAD50,mut_RAD51,mut_RAD51B,mut_RAD51C,mut_RAD51D,mut_RAD52,mut_RAD54L,mut_RAF1,mut_RARA,mut_RB1,mut_REL,mut_RET,mut_RICTOR,mut_RNF43,mut_RPTOR,mut_RUNX1,mut_SETD2,mut_SF3B1,mut_SH2B3,mut_SMAD2,mut_SMAD4,mut_SMARCA4,mut_SMARCB1,mut_SMARCD1,mut_SMO,mut_SOCS1,mut_SOX2,mut_SPEN,mut_SPOP,mut_SRC,mut_STAG2,mut_STK11,mut_SUFU,mut_SYK,mut_TBX3,mut_TET2,mut_TGFBR2,mut_TNFAIP3,mut_TNFRSF14,mut_TOP1,mut_TP53,mut_TSC1,mut_TSC2,mut_TSHR,mut_VHL,mut_WT1,mut_XPO1,cna_ABL1,cna_AKT1,cna_AKT2,cna_AKT3,cna_ALK,cna_ALOX12B,cna_AMER1,cna_APC,cna_AR,cna_ARAF,cna_ARID1A,cna_ARID2,cna_ASXL1,cna_ATM,cna_ATR,cna_ATRX,cna_AURKA,cna_AURKB,cna_AXL,cna_BAP1,cna_BARD1,cna_BCL2,cna_BCL6,cna_BCOR,cna_BLM,cna_BRAF,cna_BRCA1,cna_BRCA2,cna_BRIP1,cna_BTK,cna_CARD11,cna_CASP8,cna_CBFB,cna_CBL,cna_CCND1,cna_CCND2,cna_CCND3,cna_CCNE1,cna_CD79A,cna_CD79B,cna_CDC73,cna_CDH1,cna_CDK12,cna_CDK4,cna_CDK6,cna_CDK8,cna_CDKN1B,cna_CDKN2A,cna_CDKN2B,cna_CDKN2C,cna_CEBPA,cna_CHEK1,cna_CHEK2,cna_CIC,cna_CREBBP,cna_CRKL,cna_CRLF2,cna_CSF1R,cna_CTCF,cna_CTNNB1,cna_DAXX,cna_DDR2,cna_DIS3,cna_DNMT3A,cna_DOT1L,cna_EGFR,cna_EP300,cna_EPHA3,cna_EPHA5,cna_EPHB1,cna_ERBB2,cna_ERBB3,cna_ERBB4,cna_ERG,cna_ESR1,cna_EZH2,cna_FANCA,cna_FANCC,cna_FBXW7,cna_FGF19,cna_FGF3,cna_FGF4,cna_FGFR1,cna_FGFR2,cna_FGFR3,cna_FGFR4,cna_FLT1,cna_FLT3,cna_FLT4,cna_FOXL2,cna_GATA1,cna_GATA2,cna_GATA3,cna_GNA11,cna_GNAQ,cna_GNAS,cna_GRIN2A,cna_GSK3B,cna_HGF,cna_HLA_A,cna_HRAS,cna_IDH1,cna_IDH2,cna_IGF1,cna_IGF1R,cna_IGF2,cna_IKBKE,cna_IKZF1,cna_IL7R,cna_INHA,cna_INHBA,cna_IRF4,cna_IRS2,cna_JAK1,cna_JAK2,cna_JAK3,cna_JUN,cna_KDM5A,cna_KDM5C,cna_KDM6A,cna_KDR,cna_KEAP1,cna_KIT,cna_KMT2A,cna_KMT2D,cna_KRAS,cna_LMO1,cna_MAP2K1,cna_MAP2K2,cna_MAP2K4,cna_MAP3K1,cna_MAP3K13,cna_MCL1,cna_MDM2,cna_MDM4,cna_MED12,cna_MEF2B,cna_MEN1,cna_MET,cna_MITF,cna_MLH1,cna_MPL,cna_MRE11A,cna_MSH2,cna_MSH6,cna_MTOR,cna_MUTYH,cna_MYC,cna_MYCL,cna_MYCN,cna_MYD88,cna_NBN,cna_NCOR1,cna_NF1,cna_NF2,cna_NFE2L2,cna_NFKBIA,cna_NKX2_1,cna_NOTCH1,cna_NOTCH2,cna_NOTCH3,cna_NOTCH4,

In [11]:
# change -1 labels
#df.replace(-1 , 10)

# each row is a patient

# 15 Cancer Types
Lung, Breast, Colon, Central_Nervous_System, Pancreas, Ovary, Prostate, Uterus, Kidney, Head_and_Neck, Stomch, Bladder, Liver, Skin, Thyroid

# mut

# cna

# CIN

# destination

In [12]:
display(DataFrameSummary(df).summary())

,Lung,Breast,Colon,Central_Nervous_System,Pancreas,Ovary,Prostate,Uterus,Kidney,Head_and_Neck,Stomach,Bladder,Liver,Skin,Thyroid,mut_ABL1,mut_AKT1,mut_AKT2,mut_AKT3,mut_ALK,mut_ALOX12B,mut_AMER1,mut_APC,mut_AR,mut_ARAF,mut_ARID1A,mut_ARID2,mut_ASXL1,mut_ATM,mut_ATR,mut_ATRX,mut_AURKA,mut_AURKB,mut_AXL,mut_BAP1,mut_BARD1,mut_BCL2,mut_BCL6,mut_BCOR,mut_BLM,mut_BRAF,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_BTK,mut_CARD11,mut_CASP8,mut_CBFB,mut_CBL,mut_CCND1,mut_CCND2,mut_CCND3,mut_CCNE1,mut_CD79A,mut_CD79B,mut_CDC73,mut_CDH1,mut_CDK12,mut_CDK4,mut_CDK6,mut_CDK8,mut_CDKN1B,mut_CDKN2A,mut_CDKN2B,mut_CDKN2C,mut_CEBPA,mut_CHEK1,mut_CHEK2,mut_CIC,mut_CREBBP,mut_CRKL,mut_CRLF2,mut_CSF1R,mut_CTCF,mut_CTNNB1,mut_DAXX,mut_DDR2,mut_DIS3,mut_DNMT3A,mut_DOT1L,mut_EGFR,mut_EP300,mut_EPHA3,mut_EPHA5,mut_EPHB1,mut_ERBB2,mut_ERBB3,mut_ERBB4,mut_ERG,mut_ESR1,mut_EZH2,mut_FAM46C,mut_FANCA,mut_FANCC,mut_FBXW7,mut_FGF19,mut_FGF3,mut_FGF4,mut_FGFR1,mut_FGFR2,mut_FGFR3,mut_FGFR4,mut_FIP1L1,mut_FLT1,mut_FLT3,mut_FLT4,mut_FOXL2,mut_GATA1,mut_GATA2,mut_GATA3,mut_GNA11,mut_GNAQ,mut_GNAS,mut_GRIN2A,mut_GSK3B,mut_HGF,mut_HLA_A,mut_HRAS,mut_IDH1,mut_IDH2,mut_IGF1,mut_IGF1R,mut_IGF2,mut_IKBKE,mut_IKZF1,mut_IL7R,mut_INHBA,mut_INSRR,mut_IRF4,mut_IRS2,mut_JAK1,mut_JAK2,mut_JAK3,mut_JUN,mut_KDM5A,mut_KDM5C,mut_KDM6A,mut_KDR,mut_KEAP1,mut_KIT,mut_KMT2A,mut_KMT2D,mut_KRAS,mut_LMO1,mut_MAP2K1,mut_MAP2K2,mut_MAP2K4,mut_MAP3K1,mut_MAP3K13,mut_MCL1,mut_MDM2,mut_MDM4,mut_MED12,mut_MEF2B,mut_MEN1,mut_MET,mut_MITF,mut_MLH1,mut_MPL,mut_MRE11A,mut_MSH2,mut_MSH6,mut_MTOR,mut_MUTYH,mut_MYC,mut_MYCL,mut_MYCN,mut_MYD88,mut_NBN,mut_NCOR1,mut_NF1,mut_NF2,mut_NFE2L2,mut_NFKBIA,mut_NKX2_1,mut_NOTCH1,mut_NOTCH2,mut_NOTCH3,mut_NOTCH4,mut_NPM1,mut_NRAS,mut_NSD1,mut_NTRK1,mut_NTRK2,mut_NTRK3,mut_NUP93,mut_PAK7,mut_PALB2,mut_PARP1,mut_PAX5,mut_PBRM1,mut_PDGFRA,mut_PDGFRB,mut_PDPK1,mut_PIK3C2G,mut_PIK3C3,mut_PIK3CA,mut_PIK3CG,mut_PIK3R1,mut_PIK3R2,mut_PMS2,mut_PNRC1,mut_PPP2R1A,mut_PRDM1,mut_PRKAR1A,mut_PTCH1,mut_PTEN,mut_PTPN11,mut_RAD50,mut_RAD51,mut_RAD51B,mut_RAD51C,mut_RAD51D,mut_RAD52,mut_RAD54L,mut_RAF1,mut_RARA,mut_RB1,mut_REL,mut_RET,mut_RICTOR,mut_RNF43,mut_RPTOR,mut_RUNX1,mut_SETD2,mut_SF3B1,mut_SH2B3,mut_SMAD2,mut_SMAD4,mut_SMARCA4,mut_SMARCB1,mut_SMARCD1,mut_SMO,mut_SOCS1,mut_SOX2,mut_SPEN,mut_SPOP,mut_SRC,mut_STAG2,mut_STK11,mut_SUFU,mut_SYK,mut_TBX3,mut_TET2,mut_TGFBR2,mut_TNFAIP3,mut_TNFRSF14,mut_TOP1,mut_TP53,mut_TSC1,mut_TSC2,mut_TSHR,mut_VHL,mut_WT1,mut_XPO1,cna_ABL1,cna_AKT1,cna_AKT2,cna_AKT3,cna_ALK,cna_ALOX12B,cna_AMER1,cna_APC,cna_AR,cna_ARAF,cna_ARID1A,cna_ARID2,cna_ASXL1,cna_ATM,cna_ATR,cna_ATRX,cna_AURKA,cna_AURKB,cna_AXL,cna_BAP1,cna_BARD1,cna_BCL2,cna_BCL6,cna_BCOR,cna_BLM,cna_BRAF,cna_BRCA1,cna_BRCA2,cna_BRIP1,cna_BTK,cna_CARD11,cna_CASP8,cna_CBFB,cna_CBL,cna_CCND1,cna_CCND2,cna_CCND3,cna_CCNE1,cna_CD79A,cna_CD79B,cna_CDC73,cna_CDH1,cna_CDK12,cna_CDK4,cna_CDK6,cna_CDK8,cna_CDKN1B,cna_CDKN2A,cna_CDKN2B,cna_CDKN2C,cna_CEBPA,cna_CHEK1,cna_CHEK2,cna_CIC,cna_CREBBP,cna_CRKL,cna_CRLF2,cna_CSF1R,cna_CTCF,cna_CTNNB1,cna_DAXX,cna_DDR2,cna_DIS3,cna_DNMT3A,cna_DOT1L,cna_EGFR,cna_EP300,cna_EPHA3,cna_EPHA5,cna_EPHB1,cna_ERBB2,cna_ERBB3,cna_ERBB4,cna_ERG,cna_ESR1,cna_EZH2,cna_FANCA,cna_FANCC,cna_FBXW7,cna_FGF19,cna_FGF3,cna_FGF4,cna_FGFR1,cna_FGFR2,cna_FGFR3,cna_FGFR4,cna_FLT1,cna_FLT3,cna_FLT4,cna_FOXL2,cna_GATA1,cna_GATA2,cna_GATA3,cna_GNA11,cna_GNAQ,cna_GNAS,cna_GRIN2A,cna_GSK3B,cna_HGF,cna_HLA_A,cna_HRAS,cna_IDH1,cna_IDH2,cna_IGF1,cna_IGF1R,cna_IGF2,cna_IKBKE,cna_IKZF1,cna_IL7R,cna_INHA,cna_INHBA,cna_IRF4,cna_IRS2,cna_JAK1,cna_JAK2,cna_JAK3,cna_JUN,cna_KDM5A,cna_KDM5C,cna_KDM6A,cna_KDR,cna_KEAP1,cna_KIT,cna_KMT2A,cna_KMT2D,cna_KRAS,cna_LMO1,cna_MAP2K1,cna_MAP2K2,cna_MAP2K4,cna_MAP3K1,cna_MAP3K13,cna_MCL1,cna_MDM2,cna_MDM4,cna_MED12,cna_MEF2B,cna_MEN1,cna_MET,cna_MITF,cna_MLH1,cna_MPL,cna_MRE11A,cna_MSH2,cna_MSH6,cna_MTOR,cna_MUTYH,cna_MYC,cna_MYCL,cna_MYCN,cna_MYD88,cna_NBN,cna_NCOR1,cna_NF1,cna_NF2,cna_NFE2L2,cna_NFKBIA,cna_NKX2_1,cna_NOTCH1,cna_NOTCH2,cna_NOTCH3,cna_NOTCH4,

In [13]:
df.columns

Index(['Lung', 'Breast', 'Colon', 'Central_Nervous_System', 'Pancreas',
       'Ovary', 'Prostate', 'Uterus', 'Kidney', 'Head_and_Neck',
       ...
       'cna_TOP1', 'cna_TP53', 'cna_TSC1', 'cna_TSC2', 'cna_TSHR', 'cna_VHL',
       'cna_WT1', 'cna_XPO1', 'CIN', 'destination'],
      dtype='object', length=494)

In [ ]:
#df.drop(df.iloc[:, 30:450], inplace=True, axis=1)

In [14]:
#df.columns

In [15]:
df.head().T.head(500)

,0,1,2,3,4
Lung,0,0,0,0,1
Breast,1,0,0,0,0
Colon,0,0,0,0,0
Central_Nervous_System,0,0,0,0,0
Pancreas,0,0,0,0,0
Ovary,0,0,0,0,0
Prostate,0,0,0,0,0
Uterus,0,1,1,1,0
Kidney,0,0,0,0,0
Head_and_Neck,0,0,0,0,0


In [16]:
cat_vars = [col for col in df.columns if col not in ['destination']]
cat_vars

['Lung',
 'Breast',
 'Colon',
 'Central_Nervous_System',
 'Pancreas',
 'Ovary',
 'Prostate',
 'Uterus',
 'Kidney',
 'Head_and_Neck',
 'Stomach',
 'Bladder',
 'Liver',
 'Skin',
 'Thyroid',
 'mut_ABL1',
 'mut_AKT1',
 'mut_AKT2',
 'mut_AKT3',
 'mut_ALK',
 'mut_ALOX12B',
 'mut_AMER1',
 'mut_APC',
 'mut_AR',
 'mut_ARAF',
 'mut_ARID1A',
 'mut_ARID2',
 'mut_ASXL1',
 'mut_ATM',
 'mut_ATR',
 'mut_ATRX',
 'mut_AURKA',
 'mut_AURKB',
 'mut_AXL',
 'mut_BAP1',
 'mut_BARD1',
 'mut_BCL2',
 'mut_BCL6',
 'mut_BCOR',
 'mut_BLM',
 'mut_BRAF',
 'mut_BRCA1',
 'mut_BRCA2',
 'mut_BRIP1',
 'mut_BTK',
 'mut_CARD11',
 'mut_CASP8',
 'mut_CBFB',
 'mut_CBL',
 'mut_CCND1',
 'mut_CCND2',
 'mut_CCND3',
 'mut_CCNE1',
 'mut_CD79A',
 'mut_CD79B',
 'mut_CDC73',
 'mut_CDH1',
 'mut_CDK12',
 'mut_CDK4',
 'mut_CDK6',
 'mut_CDK8',
 'mut_CDKN1B',
 'mut_CDKN2A',
 'mut_CDKN2B',
 'mut_CDKN2C',
 'mut_CEBPA',
 'mut_CHEK1',
 'mut_CHEK2',
 'mut_CIC',
 'mut_CREBBP',
 'mut_CRKL',
 'mut_CRLF2',
 'mut_CSF1R',
 'mut_CTCF',
 'mut_CTNNB1',
 

In [17]:
dep = 'destination'
data_df = df[cat_vars + [dep]].copy()

In [18]:
for v in cat_vars: data_df[v] = data_df[v].astype('category').cat.as_ordered()

In [19]:
n = len(data_df) ; n

9317

In [20]:
idxs = get_cv_idxs(n, val_pct=1000/n)
samp = df.iloc[idxs]
samp_size = len(samp)
samp.head()

,Lung,Breast,Colon,Central_Nervous_System,Pancreas,Ovary,Prostate,Uterus,Kidney,Head_and_Neck,Stomach,Bladder,Liver,Skin,Thyroid,mut_ABL1,mut_AKT1,mut_AKT2,mut_AKT3,mut_ALK,mut_ALOX12B,mut_AMER1,mut_APC,mut_AR,mut_ARAF,mut_ARID1A,mut_ARID2,mut_ASXL1,mut_ATM,mut_ATR,mut_ATRX,mut_AURKA,mut_AURKB,mut_AXL,mut_BAP1,mut_BARD1,mut_BCL2,mut_BCL6,mut_BCOR,mut_BLM,mut_BRAF,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_BTK,mut_CARD11,mut_CASP8,mut_CBFB,mut_CBL,mut_CCND1,mut_CCND2,mut_CCND3,mut_CCNE1,mut_CD79A,mut_CD79B,mut_CDC73,mut_CDH1,mut_CDK12,mut_CDK4,mut_CDK6,mut_CDK8,mut_CDKN1B,mut_CDKN2A,mut_CDKN2B,mut_CDKN2C,mut_CEBPA,mut_CHEK1,mut_CHEK2,mut_CIC,mut_CREBBP,mut_CRKL,mut_CRLF2,mut_CSF1R,mut_CTCF,mut_CTNNB1,mut_DAXX,mut_DDR2,mut_DIS3,mut_DNMT3A,mut_DOT1L,mut_EGFR,mut_EP300,mut_EPHA3,mut_EPHA5,mut_EPHB1,mut_ERBB2,mut_ERBB3,mut_ERBB4,mut_ERG,mut_ESR1,mut_EZH2,mut_FAM46C,mut_FANCA,mut_FANCC,mut_FBXW7,mut_FGF19,mut_FGF3,mut_FGF4,mut_FGFR1,mut_FGFR2,mut_FGFR3,mut_FGFR4,mut_FIP1L1,mut_FLT1,mut_FLT3,mut_FLT4,mut_FOXL2,mut_GATA1,mut_GATA2,mut_GATA3,mut_GNA11,mut_GNAQ,mut_GNAS,mut_GRIN2A,mut_GSK3B,mut_HGF,mut_HLA_A,mut_HRAS,mut_IDH1,mut_IDH2,mut_IGF1,mut_IGF1R,mut_IGF2,mut_IKBKE,mut_IKZF1,mut_IL7R,mut_INHBA,mut_INSRR,mut_IRF4,mut_IRS2,mut_JAK1,mut_JAK2,mut_JAK3,mut_JUN,mut_KDM5A,mut_KDM5C,mut_KDM6A,mut_KDR,mut_KEAP1,mut_KIT,mut_KMT2A,mut_KMT2D,mut_KRAS,mut_LMO1,mut_MAP2K1,mut_MAP2K2,mut_MAP2K4,mut_MAP3K1,mut_MAP3K13,mut_MCL1,mut_MDM2,mut_MDM4,mut_MED12,mut_MEF2B,mut_MEN1,mut_MET,mut_MITF,mut_MLH1,mut_MPL,mut_MRE11A,mut_MSH2,mut_MSH6,mut_MTOR,mut_MUTYH,mut_MYC,mut_MYCL,mut_MYCN,mut_MYD88,mut_NBN,mut_NCOR1,mut_NF1,mut_NF2,mut_NFE2L2,mut_NFKBIA,mut_NKX2_1,mut_NOTCH1,mut_NOTCH2,mut_NOTCH3,mut_NOTCH4,mut_NPM1,mut_NRAS,mut_NSD1,mut_NTRK1,mut_NTRK2,mut_NTRK3,mut_NUP93,mut_PAK7,mut_PALB2,mut_PARP1,mut_PAX5,mut_PBRM1,mut_PDGFRA,mut_PDGFRB,mut_PDPK1,mut_PIK3C2G,mut_PIK3C3,mut_PIK3CA,mut_PIK3CG,mut_PIK3R1,mut_PIK3R2,mut_PMS2,mut_PNRC1,mut_PPP2R1A,mut_PRDM1,mut_PRKAR1A,mut_PTCH1,mut_PTEN,mut_PTPN11,mut_RAD50,mut_RAD51,mut_RAD51B,mut_RAD51C,mut_RAD51D,mut_RAD52,mut_RAD54L,mut_RAF1,mut_RARA,mut_RB1,mut_REL,mut_RET,mut_RICTOR,mut_RNF43,mut_RPTOR,mut_RUNX1,mut_SETD2,mut_SF3B1,mut_SH2B3,mut_SMAD2,mut_SMAD4,mut_SMARCA4,mut_SMARCB1,mut_SMARCD1,mut_SMO,mut_SOCS1,mut_SOX2,mut_SPEN,mut_SPOP,mut_SRC,mut_STAG2,mut_STK11,mut_SUFU,mut_SYK,mut_TBX3,mut_TET2,mut_TGFBR2,mut_TNFAIP3,mut_TNFRSF14,mut_TOP1,mut_TP53,mut_TSC1,mut_TSC2,mut_TSHR,mut_VHL,mut_WT1,mut_XPO1,cna_ABL1,cna_AKT1,cna_AKT2,cna_AKT3,cna_ALK,cna_ALOX12B,cna_AMER1,cna_APC,cna_AR,cna_ARAF,cna_ARID1A,cna_ARID2,cna_ASXL1,cna_ATM,cna_ATR,cna_ATRX,cna_AURKA,cna_AURKB,cna_AXL,cna_BAP1,cna_BARD1,cna_BCL2,cna_BCL6,cna_BCOR,cna_BLM,cna_BRAF,cna_BRCA1,cna_BRCA2,cna_BRIP1,cna_BTK,cna_CARD11,cna_CASP8,cna_CBFB,cna_CBL,cna_CCND1,cna_CCND2,cna_CCND3,cna_CCNE1,cna_CD79A,cna_CD79B,cna_CDC73,cna_CDH1,cna_CDK12,cna_CDK4,cna_CDK6,cna_CDK8,cna_CDKN1B,cna_CDKN2A,cna_CDKN2B,cna_CDKN2C,cna_CEBPA,cna_CHEK1,cna_CHEK2,cna_CIC,cna_CREBBP,cna_CRKL,cna_CRLF2,cna_CSF1R,cna_CTCF,cna_CTNNB1,cna_DAXX,cna_DDR2,cna_DIS3,cna_DNMT3A,cna_DOT1L,cna_EGFR,cna_EP300,cna_EPHA3,cna_EPHA5,cna_EPHB1,cna_ERBB2,cna_ERBB3,cna_ERBB4,cna_ERG,cna_ESR1,cna_EZH2,cna_FANCA,cna_FANCC,cna_FBXW7,cna_FGF19,cna_FGF3,cna_FGF4,cna_FGFR1,cna_FGFR2,cna_FGFR3,cna_FGFR4,cna_FLT1,cna_FLT3,cna_FLT4,cna_FOXL2,cna_GATA1,cna_GATA2,cna_GATA3,cna_GNA11,cna_GNAQ,cna_GNAS,cna_GRIN2A,cna_GSK3B,cna_HGF,cna_HLA_A,cna_HRAS,cna_IDH1,cna_IDH2,cna_IGF1,cna_IGF1R,cna_IGF2,cna_IKBKE,cna_IKZF1,cna_IL7R,cna_INHA,cna_INHBA,cna_IRF4,cna_IRS2,cna_JAK1,cna_JAK2,cna_JAK3,cna_JUN,cna_KDM5A,cna_KDM5C,cna_KDM6A,cna_KDR,cna_KEAP1,cna_KIT,cna_KMT2A,cna_KMT2D,cna_KRAS,cna_LMO1,cna_MAP2K1,cna_MAP2K2,cna_MAP2K4,cna_MAP3K1,cna_MAP3K13,cna_MCL1,cna_MDM2,cna_MDM4,cna_MED12,cna_MEF2B,cna_MEN1,cna_MET,cna_MITF,cna_MLH1,cna_MPL,cna_MRE11A,cna_MSH2,cna_MSH6,cna_MTOR,cna_MUTYH,cna_MYC,cna_MYCL,cna_MYCN,cna_MYD88,cna_NBN,cna_NCOR1,cna_NF1,cna_NF2,cna_NFE2L2,cna_NFKBIA,cna_NKX2_1,cna_NOTCH1,cna_NOTCH2,cna_NOTCH3,cna_NOTCH4,

In [21]:
samp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1512 to 7295
Columns: 494 entries, Lung to destination
dtypes: int64(494)
memory usage: 3.8 MB


In [23]:
cat_sz = [(c, len(data_df[c].cat.categories) + 1) for c in cat_vars]

In [24]:
cat_sz

[('Lung', 3),
 ('Breast', 3),
 ('Colon', 3),
 ('Central_Nervous_System', 3),
 ('Pancreas', 3),
 ('Ovary', 3),
 ('Prostate', 3),
 ('Uterus', 3),
 ('Kidney', 3),
 ('Head_and_Neck', 3),
 ('Stomach', 3),
 ('Bladder', 3),
 ('Liver', 3),
 ('Skin', 3),
 ('Thyroid', 3),
 ('mut_ABL1', 3),
 ('mut_AKT1', 3),
 ('mut_AKT2', 3),
 ('mut_AKT3', 3),
 ('mut_ALK', 3),
 ('mut_ALOX12B', 3),
 ('mut_AMER1', 3),
 ('mut_APC', 3),
 ('mut_AR', 3),
 ('mut_ARAF', 3),
 ('mut_ARID1A', 3),
 ('mut_ARID2', 3),
 ('mut_ASXL1', 3),
 ('mut_ATM', 3),
 ('mut_ATR', 3),
 ('mut_ATRX', 3),
 ('mut_AURKA', 3),
 ('mut_AURKB', 3),
 ('mut_AXL', 3),
 ('mut_BAP1', 3),
 ('mut_BARD1', 3),
 ('mut_BCL2', 3),
 ('mut_BCL6', 3),
 ('mut_BCOR', 3),
 ('mut_BLM', 3),
 ('mut_BRAF', 3),
 ('mut_BRCA1', 3),
 ('mut_BRCA2', 3),
 ('mut_BRIP1', 3),
 ('mut_BTK', 3),
 ('mut_CARD11', 3),
 ('mut_CASP8', 3),
 ('mut_CBFB', 3),
 ('mut_CBL', 3),
 ('mut_CCND1', 3),
 ('mut_CCND2', 3),
 ('mut_CCND3', 3),
 ('mut_CCNE1', 3),
 ('mut_CD79A', 3),
 ('mut_CD79B', 3),
 ('m

In [25]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

[(3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 

In [26]:
model = MixedInputModel(emb_szs, n_cont=0, emb_drop=0, out_sz=2, szs=[500], drops=[0.5]).cuda()

In [27]:
bm = BasicModel(model, 'multiclass_classifier')

In [28]:
train_ratio = 0.75
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))
val_idx[0]

750

In [ ]:
md = ColumnarModelData.from_data_frames(PATH, )

In [ ]:
df, y, nas, mapper = proc_df(samp, 'destination', do_scale=True)

In [ ]:
df.head()

In [ ]:
y

In [ ]:
nas

In [ ]:
yl = np.log(y)

In [ ]:
yl

In [ ]:
#df_test, _, nas, mapper = proc_df()

In [ ]:
train_ratio = 0.75
train_size = int(samp_size * train_ratio); train_size

In [ ]:
val_idx = list(range(train_size, len(df)))
val_idx[0]

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [ ]:
max_log_y

In [ ]:
yl

In [ ]:
y_range

In [ ]:
#??ColumnarModelData

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128)

In [ ]:
cat_sz = [(c, len(data_df[c].cat.categories) + 1) for c in cat_vars]

In [ ]:
#cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _, c in cat_sz]

In [ ]:
#emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
df.head()

In [ ]:
print(m)

In [ ]:
m.fit(lr, 1, metrics=[exp_rmspe], cycle_len=1)